# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time
import numpy as np
import socket

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.lasers import wlm_monitor
from pylabnet.network.client_server import sweeper, si_tt_cnt_monitor
from pylabnet.utils.sweeper import Sweep1D, MultiChSweep1D
from pylabnet.utils.helper_methods import create_server

Enter host and port parameters here:

In [17]:
logger_host = '140.247.189.94'
logger_port = 21379
wavemeter_host = '140.247.189.94'
wavemeter_port = 6668
ctr_host = '140.247.189.82'
ctr_port = 47717

Instantiate all clients

In [18]:
exp_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
wm = wlm_monitor.Client(
    host=wavemeter_host,
    port=wavemeter_port,
)
ctr = si_tt_cnt_monitor.Client(
    host=ctr_host,
    port=ctr_port
)
ctr.set_channels(name='sweeper', ch_list=[1])
ctr.init_rate_monitor(name='sweeper')
exp = Sweep1D(logger=exp_log)

In [19]:
update_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='1D_update_server'
)
pause_1d_service = sweeper.Service()
pause_1d_service.assign_module(module=exp)
pause_1d_service.assign_logger(logger=update_log)
pause_1d_server, pause_1d_port = create_server(
    service=pause_1d_service,
    logger=update_log,
    host=socket.gethostbyname(socket.gethostname())
)
update_log.update_data(data={'port': pause_1d_port})
pause_1d_server.start()

## Experiment

Parameters, setup

In [20]:
f0 = 406.700
f1 = 406.702
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts)

array([72623.17657325])

Experiment script

In [21]:
def experiment(frequency):

    # Set frequency
    wm.update_parameters([{
        'channel': 6, 'setpoint': frequency
    }])

    return ctr.get_count_rate(name='sweeper', integration=0.1)
    

In [22]:
exp.configure_experiment(experiment)

In [25]:
exp.set_reps(reps=3)
exp.run(plot=True, autosave=False)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'b33685aa-e52f-49d2-9c38-0af9917e5e14',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'ea42a4a0-bfd6-4dd1-b800-ae5bb94e59de',
 …

## Multi-channel sweep

In [9]:
mc_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop_multichannel'
)

mc_pause_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='MultiCh1D_pause_server'
)

m_exp = MultiChSweep1D(logger=mc_log, channels=['Transmission', 'Reflection'])
pause_m_service = sweeper.Service()
pause_m_service.assign_module(module=exp)
pause_m_service.assign_logger(logger=mc_pause_log)
pause_m_server = create_server(
    service=pause_m_service,
    logger=mc_pause_log,
    host=logger_host
)

In [18]:
m_exp.set_parameters(min=f0, max=f1, pts=pts, reps=4)

In [19]:
def multi_experiment(frequency):
    time.sleep(0.02)
    diffusion = 0.0002*np.sin(2*np.pi*time.time()/50)
    return [
        100 + 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2),
        100 - 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2)
    ]
m_exp.configure_experiment(multi_experiment)

In [20]:
m_exp.run(plot=True)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'a36df67b-3158-49db-a510-0af4474ca83f',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '9edc977a-61b7-4b8e-ba4b-46b5c4b7e1f6',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '1825d0c6-bb91-4049-97b0-de0e9d2c5364',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'b8187589-e1e3-46f5-ba41-4f153018ad26',
 …

## Close counter

In [5]:
#ni.close_timed_counter('counter_0')